In [16]:
from indra import literature 
import numpy as np
import pymysql
import pandas as pd
from bs4 import BeautifulSoup, Comment
import requests

In [102]:
"""
function return dict gene Entrez 2 Symbols
"""
def genent2sym():

    db = pymysql.connect("menchelabdb.int.cemm.at","readonly","ra4Roh7ohdee","GenesGO")    

    # prepare a cursor object using cursor() method
    cursor = db.cursor()

    sql = """   SELECT
                    Approved_Symbol,
                    Entrez_Gene_ID_NCBI 
                FROM GenesGO.hgnc_complete
                WHERE Entrez_Gene_ID_NCBI != ''
          """ 

    cursor.execute(sql)
    data = cursor.fetchall()    
#     try: 
#         # execute SQL query using execute() method.
#         cursor.execute(sql)
#         data = cursor.fetchall()
#     except:
#         print('SQL error')
    db.close()

#     t0 = time.time()
    d_sym_ent = {}
    d_ent_sym = {}

    for x in data:
        sym = x[0]
        ent = x[1]
        d_sym_ent[sym] = ent
        d_ent_sym[ent] = sym
#     print(time.time()-t0)
    
    return d_ent_sym, d_sym_ent



"""
function return dict pub_id 2 titles
"""
def genent2pubtitle(entrez):

    db = pymysql.connect("menchelabdb.int.cemm.at","readonly","ra4Roh7ohdee","GenesGO")    

    # prepare a cursor object using cursor() method
    cursor = db.cursor()

    sql = """   SELECT
                    tx.pubid,
                    tx.title
                FROM Datadivr.gene2pubid gp
                INNER JOIN Datadivr.pubid2text tx
                ON gp.pubid = tx.pubid
                WHERE gp.entrez = '%s'
          """ %entrez

    cursor.execute(sql)
    data = cursor.fetchall()    
#     try: 
#         # execute SQL query using execute() method.
#         cursor.execute(sql)
#         data = cursor.fetchall()
#     except:
#         print('SQL error')
    db.close()

#     t0 = time.time()
    d_pid_tlt = {}

    for x in data:
        d_pid_tlt[x[0]] = x[1]
    
    return d_pid_tlt


"""
function return dict pub_id 2 titles
"""
def pubid2authorabstract(pubid):

    db = pymysql.connect("menchelabdb.int.cemm.at","readonly","ra4Roh7ohdee","GenesGO")    

    # prepare a cursor object using cursor() method
    cursor = db.cursor()

    sql = """   SELECT
                    author,
                    abstract
                FROM Datadivr.pubid2text
                WHERE pubid = '%s'
          """ %pubid

    cursor.execute(sql)
    data = cursor.fetchall()    
#     try: 
#         # execute SQL query using execute() method.
#         cursor.execute(sql)
#         data = cursor.fetchall()
#     except:
#         print('SQL error')
    db.close()

#     t0 = time.time()
#     l_text = []

    for x in data:
        text_str = x[0] + '<br>' + x[1]
#         l_text.append(text_str)
    
    return text_str


"""
function gets search string return l_entrez
"""
def genesfromstr(searchstr):

    searchstr_mod = '%'+searchstr+'%' 
    db = pymysql.connect("menchelabdb.int.cemm.at","readonly","ra4Roh7ohdee","GenesGO")    

    # prepare a cursor object using cursor() method
    cursor = db.cursor()

    sql = """   SELECT
                DISTINCT
                gp.entrez
                #pubid
                FROM Datadivr.pubid2text tx 
                INNER JOIN Datadivr.gene2pubid gp
                ON gp.pubid = tx.pubid
                WHERE tx.title LIKE '%s'
          """ %searchstr_mod

    cursor.execute(sql)
    data = cursor.fetchall()    
#     try: 
#         # execute SQL query using execute() method.
#         cursor.execute(sql)
#         data = cursor.fetchall()
#     except:
#         print('SQL error')
    db.close()

#     t0 = time.time()
    l_entz = []

    for x in data:
#         text_str = x[0] + '<br>' + x[1]
        l_entz.append(x[0])
    
    return l_entz

# LOOK for Articles containing a search string

In [111]:
search_str = 'interactome'

prestr = 'https://www.ncbi.nlm.nih.gov/pubmed/?term='

l_pubIDs = literature.pubmed_client.get_ids(search_str)


print('Number of articles found: %s' %len(l_pubIDs))
# for pID in l_pubIDs:
#     print(prestr+pID)


Number of articles found: 3701


# -------------------------------------------------------------

# GET all PubIDS for gene and make urls and title out of it

In [116]:
entrezID = '54790'

d_ent_sym, d_sym_ent = genent2sym()
genesym = d_ent_sym[entrezID]
print(genesym)

# get list of all pubmedIDs
l_pubIDs = literature.pubmed_client.get_ids_for_gene(genesym)

l_title = []
print(l_pubIDs)
print(len(l_pubIDs))

#list of pubmed urls

prestr = 'https://www.ncbi.nlm.nih.gov/pubmed/?term='
l_urls = []
for pubID in l_pubIDs:
    l_urls.append(prestr+pubID)
    l_title.append(literature.pubmed_client.get_title(pubID))


print(l_urls)

TET2
['25040794', '20520643', '28521175', '29275866', '27997762', '20880116', '24433485', '24218139', '26404510', '25972343', '26093090', '24413737', '28077417', '28407691', '27881874', '25200248', '26980223', '25853800', '24394411', '19372255', '12646957', '22395470', '21723200', '19767753', '19262599', '26568194', '24697267', '29773648', '23535729', '20508616', '23563607', '21734233', '27729528', '19957346', '29331390', '27848178', '26917261', '26277372', '29411884', '22017486', '28167452', '22905207', '23009937', '27005421', '20595094', '19420352', '19483684', '24898826', '22080757', '22943828', '28218607', '26880370', '27819678', '20881960', '26045292', '26843622', '23827711', '27027260', '25961778', '23398456', '24994606', '20955399', '28731140', '21760590', '19564637', '12477932', '24723429', '20693430', '25381129', '26356709', '29735542', '21346257', '26524525', '26414667', '23135354', '26771811', '30045709', '21275969', '27761930', '27102770', '24122999', '21794915', '28003489'

# -------------------------------------------------------------

# GET stuff


In [ ]:
l_pubIDs = literature.pubmed_client.get_ids_for_gene(genesym)
# print(len(l_pubIDs))

# out_str = 'Number of articles linked: %s <br>' %len(l_pubIDs)

if len(l_pubIDs) >5:
    max_ids = 5
else:
    max_ids = len(l_pubIDs)
    
d_title_abstract = {}

for pubID in l_pubIDs[:max_ids]:
    d_meta = literature.pubmed_client.get_metadata_for_ids(pubID)    
    for x in d_meta.values():
        out_str += 'Title: "%s"' %x['title'] 
#         print('Title: "%s"' %x['title'])

        title = x['title']
        
        authorstr = ''
        for a in x['authors']:
            authorstr += a + ','
        authorstr = 'Authors: ' + authorstr[:-1] +'<br>'    
        
#         print('Authors: %s' %authorstr[:-1])
    
#     print(literature.pubmed_client.get_abstract(pubID,prepend_title = False))
    abstract = literature.pubmed_client.get_abstract(pubID,prepend_title = False)
    
    d_title_abstract[title] = authorstr + abstract
    
print(d_title_abstract)

In [ ]:
r = requests.get(l_urls[0])
soup = BeautifulSoup(r.content, "lxml")

for text in soup.body.find_all(string=True):
    if text.parent.name not in ['script', 'meta', 'link', 'style'] and not isinstance(text, Comment) and text != '\n':
        print(text.strip())

# WRITE TO FILE 4 DB

In [3]:
# generate file for Datadivr.gene2pubid
d_ent_sym, d_sym_ent = genent2sym()
f = open('gene2pubID.csv','w')
c = 0
l_pid = []


for entz, sym in d_ent_sym.items():
#     if c<100:

    # get pubmed IDs
    try:
        l_pubIDs = literature.pubmed_client.get_ids_for_gene(sym)
    except:
        l_pubIDs = []
    prestr = 'https://www.ncbi.nlm.nih.gov/pubmed/?term='


    for pID in l_pubIDs:
        f.write('%s,%s,%s,%s\n' %(entz,sym,pID,prestr+pID))
        l_pid.append(pID)
#     c += 1
    


In [10]:
# generate file for Datadivr.gene2pubid
d_ent_sym, d_sym_ent = genent2sym()
# f = open('gene2pubID.csv','w')
c = 0
l_pid = []
for entz, sym in d_ent_sym.items():
#     if c <100:

    # get gene symbol
    genesym = d_ent_sym[entrezID]
#         print(genesym)

    # get pubmed IDs
    l_pubIDs = literature.pubmed_client.get_ids_for_gene(genesym)
#     prestr = 'https://www.ncbi.nlm.nih.gov/pubmed/?term='


    for pID in l_pubIDs:
        l_pid.append(pID)
#         f.write('%s,%s,%s,%s\n' %(entz,sym,pID,prestr+pID))
# c += 1
    

In [12]:
print(len(set(l_pid)))

34


# READ FILES TO GENERATE TITLE, AUTHORs AND ABSTRACT FOR EACH PUB_ID

In [10]:
f = open('gene2pubID.csv','r')

lines = f.readlines()

l_pids = []
for line in lines:
    pid = line.strip().split(',')[2]
    l_pids.append(pid)
f.close()

In [38]:
f = open('uniquepubIDs.csv','r')

lines = f.readlines()

l_pids = lines[0].strip().split(',')

# print(len(set(l_pids)))

f.close()


fstr = open('pud2text.csv','w')


for pubID in list(set(l_pids))[1:10]:
    print(pubID)
    fstr.write('%s|' %pubID)
    d_meta = literature.pubmed_client.get_metadata_for_ids(pubID)    
    for x in d_meta.values():
#         out_str += 'Title: "%s"' %x['title'] 
        print('Title: "%s"' %x['title'])
        fstr.write('%s|' %x['title'])
        title = x['title']
        
        authorstr = ''
        for a in x['authors']:
            authorstr += a + ','
        authorstr = 'Authors: ' + authorstr[:-1] #+'<br>'    
#         print()
        print('Authors: %s' %authorstr[:-1])
        fstr.write('%s|' %authorstr.split(':')[1])
# #     print(literature.pubmed_client.get_abstract(pubID,prepend_title = False))
    abstract = literature.pubmed_client.get_abstract(pubID,prepend_title = False)    
    print('Abstract: %s' %abstract)
    fstr.write('%s\n' %abstract)


# #     titles = literature.pubmed_client.get_title(pubID) 
    print('\n')
#     print(pubID,title,authorstr[:-1],abstract)
    
    
    
    
fstr.close()

8854877
Title: "Mapping of the MR60/ERGIC-53 gene to human chromosome 18q21.3-18q22 by in situ hybridization."
Authors: Authors: Arar,Mignon,Mattei,Monsigny,Roche,Legran
Abstract: 


12032587
Title: "Variability of the human aryl hydrocarbon receptor nuclear translocator (ARNT) gene."
Authors: Authors: Scheel,Hussong,Schrenk,Schmit
Abstract: The aryl hydrocarbon receptor nuclear translocator (ARNT) plays an essential role in vertebrate transcriptional regulation as the common subunit of transcriptionally active complexes like the aryl hydrocarbon receptor (AHR)/ARNT heterodimer and hypoxia-inducible factor 1, mediating cellular responses to certain xenobiotics and to hypoxia, respectively. A cohort of healthy Caucasian volunteers was screened for genetic variations of ARNT. Six polymorphic sites could be identified, a variation in a G-stretch upstream of the ATG translation start site, a frequent silent mutation (G567C), two polymorphic sites in intron 9, and two single nucleotide subs

In [82]:
# check pud2text.csv

f = open('pud2text.csv','r')

lines = f.readlines()
c = 1
for line in lines:
    count_chars = len(line.split('|'))
    if count_chars != 4:
        print(c,count_chars)
    c += 1

125983 2


In [88]:
entrez = 666
d_pid_tlt = genent2pubtitle(entrez)

print(l_tlt)

{20171287: 'Voxelwise genome-wide association study (vGWAS).', 22623531: 'Functional genomics identifies therapeutic targets for MYC-driven cancer.', 15489334: 'The status, quality, and expansion of the NIH full-length cDNA project: the Mammalian Gene Collection (MGC).', 10579309: 'Characterization of the antiapoptotic Bcl-2 family member myeloid cell leukemia-1 (Mcl-1) and the stimulation of its message by gonadotropins in the rat ovary.', 16772296: 'Bok, Bcl-2-related Ovarian Killer, Is Cell Cycle-regulated and Sensitizes to Stress-induced Apoptosis.', 10811653: 'Drosophila pro-apoptotic Bcl-2/Bax homologue reveals evolutionary conservation of cell death mechanisms.'}


In [92]:
pubid = 29407229

l_str = pubid2authorabstract(pubid)

print(l_str)

 Tian,Li,Zhang,Li,Gao<br>Osteosarcoma (OS) is a common primary malignancy in children and adolescents with relative high survival rate after chemotherapy. While, the toxicity of chemotherapy and personalized different response to chemotherapy makes it difficult for the selection of therapeutics and improvement of diagnosis. In this study, we conducted a combined analysis of two types of microarray datasets (gene expression and DNA methylation) from the Gene Expression Omnibus (GEO). Differential methylation sites (DMS) were identified by the IMA package and differential expression genes (DEGs) were screened out via the limma package. A total of 11,242 DMS (corresponding to 3080 genes (DMGs)) and 337 DEGs, with 40 overlaps (OS genes) between DEGs and DMGs, were identified. Enriched functions of OS genes were obtained through the Database for Annotation, Visualization and Integrated Discovery (DAVID). The OS genes were mainly enriched in the biological processes related to inflammatory/i

# Get list of genes for a search string in article titles

In [107]:
searchstr = 'immunodeficiency'
l_entrez = genesfromstr(searchstr)

print(len(l_entrez))


634


In [115]:
symbol = 'TET2'
print(d_sym_ent[symbol])

54790
